In [24]:
import os
import torch
import gc
import logging

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()        
    torch.cuda.ipc_collect()


try:
    %run setup_paths
except:
    %run notebooks/setup_paths
    

logging.basicConfig(
    level=logging.INFO,  # or DEBUG, WARNING, etc.
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)

logging.info(f"current dir: {os.getcwd()}")

2025-08-18 22:55:45,364 - INFO - current dir: c:\Projects\scmsim


In [25]:
%run src/utils
%run src/loaders


In [26]:
## Configuration and User-Item data selection

In [27]:
cfg = read_cfg('configs/config.yaml')
paths = PathProvider(cfg['paths']['results'], cfg['paths']['products'])
uidata = MovieLensData(get_uidata_loader(cfg, 'ml-1m'))


## Create IPW Estimates for a single model

- A few variants of IPW variants on top of MF30 model
- The results would results/MovieLens1M/estimations/MoviesCausalGPT/MF30.IPW
- The model location is results/MovieLens1M/models/MF30.0 (version 0)

In [28]:
%run src/estimators
%run src/pipeline

model_name = 'MF30'
model = load_model(paths, uidata.name(), model_name)
ipw_params = [IPWParams(0, False), IPWParams(0, True), IPWParams(0.2, True)]
ipwe = MFIPWEstimator(model_name, model, ipw_params)    
create_estimations(paths, uidata, f'{model_name}.IPW', 'MoviesCausalGPT', ipwe, reset_ids=True)


2025-08-18 22:55:50,860 - INFO - loading model MF30 (version 0) at results\MovieLens1M\models\MF30.0\weights.pt
2025-08-18 22:55:50,927 - INFO - loading info
2025-08-18 22:55:51,009 - INFO - processing: 0:4096 / 8384


C:\Projects\scmsim\src\loaders.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dir_path = paths.model_dir_path(data_name, model_name, version=0)


2025-08-18 22:55:51,439 - INFO - loading ratings
2025-08-18 22:56:06,970 - INFO - processing: 4096:8192 / 8384
2025-08-18 22:56:08,400 - INFO - processing: 8192:8384 / 8384
2025-08-18 22:56:09,208 - INFO - saving estimations at results\MovieLens1M\estimations\MoviesCausalGPT\MF30.IPW


,MF30.IPW,MF30.IPW.s,MF30.IPW.clp0.2.s,treatment_idx,resp_idx
0,-0.023013,-0.000105,-0.004878,1254,1515
1,-0.028974,-0.000037,-0.005796,166,1515
2,-0.225000,-0.001024,-0.047692,1254,648
3,-0.047517,-0.005984,-0.013899,1197,648
4,-0.150828,-0.016313,-0.038982,1193,648
...,...,...,...,...,...
8379,-0.043874,-0.000197,-0.008991,107,1395
8380,-0.044205,-0.000086,-0.008994,76,1395
8381,-0.034934,-0.000630,-0.008139,736,1125
8382,-0.365894,-0.002267,-0.074972,1480,527


- Lets do the same for additional models

In [29]:
%run src/estimators
%run src/pipeline

for model_name in ['MF40', 'MF50', 'MF60']:
    model = load_model(paths, uidata.name(), model_name)
    ipwe = MFIPWEstimator(model_name, model, ipw_params)    
    ## reset_ids mean we re-match the item-ids from the title name 
    create_estimations(paths, uidata, f'{model_name}.IPW', 'MoviesCausalGPT', ipwe, reset_ids=True)



2025-08-18 22:56:20,465 - INFO - loading model MF40 (version 0) at results\MovieLens1M\models\MF40.0\weights.pt
2025-08-18 22:56:20,547 - INFO - processing: 0:4096 / 8384


C:\Projects\scmsim\src\loaders.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dir_path = paths.model_dir_path(data_name, model_name, version=0)


2025-08-18 22:56:22,480 - INFO - processing: 4096:8192 / 8384
2025-08-18 22:56:25,191 - INFO - processing: 8192:8384 / 8384
2025-08-18 22:56:26,126 - INFO - saving estimations at results\MovieLens1M\estimations\MoviesCausalGPT\MF40.IPW
2025-08-18 22:56:26,395 - INFO - loading model MF50 (version 0) at results\MovieLens1M\models\MF50.0\weights.pt


C:\Projects\scmsim\src\loaders.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dir_path = paths.model_dir_path(data_name, model_name, version=0)


2025-08-18 22:56:26,654 - INFO - processing: 0:4096 / 8384
2025-08-18 22:56:29,464 - INFO - processing: 4096:8192 / 8384
2025-08-18 22:56:32,601 - INFO - processing: 8192:8384 / 8384
2025-08-18 22:56:33,670 - INFO - saving estimations at results\MovieLens1M\estimations\MoviesCausalGPT\MF50.IPW
2025-08-18 22:56:33,896 - INFO - loading model MF60 (version 0) at results\MovieLens1M\models\MF60.0\weights.pt


C:\Projects\scmsim\src\loaders.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dir_path = paths.model_dir_path(data_name, model_name, version=0)


2025-08-18 22:56:34,167 - INFO - processing: 0:4096 / 8384
2025-08-18 22:56:37,291 - INFO - processing: 4096:8192 / 8384
2025-08-18 22:56:40,581 - INFO - processing: 8192:8384 / 8384
2025-08-18 22:56:41,503 - INFO - saving estimations at results\MovieLens1M\estimations\MoviesCausalGPT\MF60.IPW


## Basic Estimations


In [30]:
basee = BasicEstimator()
create_estimations(paths, uidata, 'Basic' , 'MoviesCausalGPT', basee, reset_ids=True)


2025-08-18 22:57:50,720 - INFO - processing: 0:4096 / 8384
2025-08-18 22:57:51,196 - INFO - processing: 4096:8192 / 8384
2025-08-18 22:57:51,618 - INFO - processing: 8192:8384 / 8384
2025-08-18 22:57:51,665 - INFO - saving estimations at results\MovieLens1M\estimations\MoviesCausalGPT\Basic


,CORR,LIFT,SATE,treatment_idx,resp_idx
0,-0.060259,2.136280,0.036812,1254,1515
1,0.198620,7.658727,0.201577,166,1515
2,-0.065741,1.493124,0.134777,1254,648
3,0.355102,1.549424,0.225409,1197,648
4,0.173428,1.238234,0.084307,1193,648
...,...,...,...,...,...
8379,0.009835,2.559322,0.079530,107,1395
8380,0.037931,2.760617,0.088601,76,1395
8381,-0.077849,2.190710,0.092987,736,1125
8382,-0.134202,1.661950,0.264092,1480,527


## Show all pair estimations

In [31]:
%run src/utils
load_all_estimations(paths, 'MovieLens1M', 'MoviesCausalGPT')

,CORR,LIFT,SATE,treatment_idx,resp_idx,MF30.IPW,MF30.IPW.s,MF30.IPW.clp0.2.s,MF40.IPW,MF40.IPW.s,MF40.IPW.clp0.2.s,MF50.IPW,MF50.IPW.s,MF50.IPW.clp0.2.s,MF60.IPW,MF60.IPW.s,MF60.IPW.clp0.2.s
0,-0.060259,2.136280,0.036812,1254,1515,-0.023013,-0.000105,-0.004878,-0.023013,-0.000118,-0.004872,-0.023013,-0.000066,-0.004838,-0.023013,-0.000070,-0.004892
1,0.198620,7.658727,0.201577,166,1515,-0.028974,-0.000037,-0.005796,-0.028974,-0.000050,-0.005805,-0.028974,-0.000037,-0.005805,-0.028974,-0.000034,-0.005809
2,-0.065741,1.493124,0.134777,1254,648,-0.225000,-0.001024,-0.047692,-0.225000,-0.001152,-0.047632,-0.225000,-0.000641,-0.047298,-0.225000,-0.000682,-0.047831
3,0.355102,1.549424,0.225409,1197,648,-0.047517,-0.005984,-0.013899,-0.047517,-0.005242,-0.014041,-0.047517,-0.003704,-0.014133,-0.047517,-0.003440,-0.014273
4,0.173428,1.238234,0.084307,1193,648,-0.150828,-0.016313,-0.038982,-0.150828,-0.017873,-0.040564,-0.150828,-0.012342,-0.038447,-0.150828,-0.016134,-0.039009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8469,0.009835,2.559322,0.079530,107,1395,-0.043874,-0.000197,-0.008991,-0.043874,-0.000187,-0.009003,-0.043874,-0.000105,-0.009052,-0.043874,-0.000093,-0.009049
8470,0.037931,2.760617,0.088601,76,1395,-0.044205,-0.000086,-0.008994,-0.044205,-0.000091,-0.008995,-0.044205,-0.000058,-0.009024,-0.044205,-0.000053,-0.008968
8471,-0.077849,2.190710,0.092986,736,1125,-0.034934,-0.000630,-0.008139,-0.034934,-0.000634,-0.008245,-0.034934,-0.000285,-0.008240,-0.034934,-0.000253,-0.008262
8472,-0.134202,1.661950,0.264092,1480,527,-0.365894,-0.002267,-0.074972,-0.365894,-0.002439,-0.074445,-0.365894,-0.001812,-0.074609,-0.365894,-0.001693,-0.074751


## Method Comparison

In [33]:


get_causal_gpt_scores(paths, uidata)

,name,corr,corr_pos,zero_mse
0,CORR,0.102903,0.166309,4.027582e-02
1,LIFT,0.149360,0.163592,1.567285e+01
2,SATE,0.156771,0.144169,3.137042e-02
3,MF30.IPW,-0.010582,0.029612,1.069278e-02
4,MF30.IPW.s,-0.010474,-0.005667,9.585546e-07
5,MF30.IPW.clp0.2.s,-0.012247,0.024118,4.615669e-04
6,MF40.IPW,-0.010582,0.029612,1.069278e-02
7,MF40.IPW.s,-0.010597,-0.008258,9.231223e-07
8,MF40.IPW.clp0.2.s,-0.012185,0.024360,4.633239e-04
9,MF50.IPW,-0.010582,0.029612,1.069278e-02
